In [1]:
from kafka import KafkaConsumer,KafkaProducer
import json,pickle

In [2]:
consumer = KafkaConsumer('predict-model', bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',  # Start from the earliest message
    enable_auto_commit=True,
    group_id='location-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8')))

model=pickle.load(open("model_final.pkl","rb"))
producer =KafkaProducer(bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8') )

def predictandsend():
    i=0
    for data in consumer:
        tweet=data.value
        typesentim=model.predict([tweet['data']['selftext']])
        if typesentim=='pos':
            print('positive')
            producer.send('stock-data',{'text':tweet['data']['selftext'],'label':'pos','id':tweet['data']['id'],'date':tweet['data']['created']})
        if typesentim=='neg':
            print('negative')
            producer.send('stock-data',{'text':tweet['data']['selftext'],'label':'neg','id':tweet['data']['id'],'date':tweet['data']['created']})
        # print(f"{tweet['data']['title'].replace("'","''")}','{tweet['data']['selftext'].replace("'","''")}'")


In [4]:

predictandsend()

positive
negative
positive
positive


KeyboardInterrupt: 